## `Procedures`

In [1]:
from sqlalchemy.engine import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text

In [2]:
db_user = 'root'
db_password = 'rushi12345'
db_host = '192.168.29.7'
db_name = 'data_prac_db' 

engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
SessionLocal = sessionmaker(autoflush=True, bind=engine)

In [3]:
db = SessionLocal()

In [4]:
db.execute(text("""
show tables;
""")).fetchmany()

[('bank_details',)]

In [19]:
db.execute(text("""
    CREATE PROCEDURE head(IN param INT)
    BEGIN
        SELECT * FROM bank_details LIMIT param;
    END
"""))

In [21]:
db.execute(text("""
call head(5)
""")).fetchall()

[(58, 'management', 'married', 'tertiary', 'no', 2143, 'yes', 'no', 'unknown', 5, 'may', 261, 1, -1, 0, 'unknown', 'no'),
 (44, 'technician', 'single', 'secondary', 'no', 29, 'yes', 'no', 'unknown', 5, 'may', 151, 1, -1, 0, 'unknown', 'no'),
 (33, 'entrepreneur', 'married', 'secondary', 'no', 2, 'yes', 'yes', 'unknown', 5, 'may', 76, 1, -1, 0, 'unknown', 'no'),
 (47, 'blue-collar', 'married', 'unknown', 'no', 1506, 'yes', 'no', 'unknown', 5, 'may', 92, 1, -1, 0, 'unknown', 'no'),
 (33, 'unknown', 'single', 'unknown', 'no', 1, 'no', 'no', 'unknown', 5, 'may', 198, 1, -1, 0, 'unknown', 'no')]

In [22]:
db.execute(text("""
select * from bank_details where balance in (select max(balance) from bank_details);
""")).fetchone()

(51, 'management', 'single', 'tertiary', 'no', 102127, 'no', 'no', 'cellular', 3, 'jun', 90, 1, -1, 0, 'unknown', 'no')

In [8]:
db.execute(text("""
CREATE PROCEDURE getAvgByJob(IN inputparam varchar(32))
BEGIN
  SELECT AVG(balance) FROM bank_details WHERE job = inputparam;
END
"""))

In [23]:
db.execute(text("""
call getAvgByJob("admin.")
""")).fetchone()

(Decimal('1135.8389'),)

In [24]:
db.execute(text("""
call getAvgByJob("retired");
""")).fetchone()

(Decimal('1984.2151'),)

In [25]:
db.execute(text("""
call getAvgByJob("unknown");
""")).fetchone()

(Decimal('1772.3576'),)

In [28]:
db.execute(text("""
CREATE PROCEDURE getRecordsbyEducationAndJob(IN param1 varchar(32), IN param2 varchar(32))
BEGIN
  SELECT * FROM bank_details WHERE education = param1 and job = param2;
END
"""))

In [29]:
db.execute(text("""
call getRecordsbyEducationAndJob("primary", "unknown");
""")).fetchall()

[(29, 'unknown', 'single', 'primary', 'no', 50, 'yes', 'no', 'unknown', 7, 'may', 50, 2, -1, 0, 'unknown', 'no'),
 (41, 'unknown', 'single', 'primary', 'no', 2398, 'yes', 'no', 'unknown', 8, 'may', 287, 2, -1, 0, 'unknown', 'no'),
 (49, 'unknown', 'married', 'primary', 'no', 341, 'yes', 'yes', 'unknown', 15, 'may', 520, 2, -1, 0, 'unknown', 'yes'),
 (35, 'unknown', 'divorced', 'primary', 'no', 3888, 'no', 'no', 'unknown', 26, 'may', 422, 5, -1, 0, 'unknown', 'no'),
 (41, 'unknown', 'married', 'primary', 'no', 526, 'no', 'no', 'unknown', 26, 'may', 151, 3, -1, 0, 'unknown', 'no'),
 (30, 'unknown', 'single', 'primary', 'no', 13052, 'no', 'no', 'unknown', 4, 'jun', 140, 1, -1, 0, 'unknown', 'no'),
 (43, 'unknown', 'single', 'primary', 'no', 433, 'no', 'no', 'unknown', 4, 'jun', 45, 1, -1, 0, 'unknown', 'no'),
 (52, 'unknown', 'married', 'primary', 'no', 8251, 'no', 'no', 'unknown', 5, 'jun', 397, 1, -1, 0, 'unknown', 'no'),
 (59, 'unknown', 'married', 'primary', 'no', 1470, 'no', 'no', 'u

In [30]:
db.execute(text("""
call head(5)
""")).keys()

RMKeyView(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y'])

In [31]:
db.execute(text("""
create procedure getRecordsByLoan(IN param varchar(32))
begin
  select * from bank_details where loan = param;
end
"""))

In [39]:
db.execute(text("""
select count(*) from bank_details where loan = "no"
""")).fetchall()

[(37967,)]

In [45]:
db.execute(text("""
create view bank_view as select age, job, marital, balance, education from bank_details;
"""))

In [48]:
db.execute(text("""
select * from bank_view limit 10;
""")).fetchall()

[(58, 'management', 'married', 2143, 'tertiary'),
 (44, 'technician', 'single', 29, 'secondary'),
 (33, 'entrepreneur', 'married', 2, 'secondary'),
 (47, 'blue-collar', 'married', 1506, 'unknown'),
 (33, 'unknown', 'single', 1, 'unknown'),
 (35, 'management', 'married', 231, 'tertiary'),
 (28, 'management', 'single', 447, 'tertiary'),
 (42, 'entrepreneur', 'divorced', 2, 'tertiary'),
 (58, 'retired', 'married', 121, 'primary'),
 (43, 'technician', 'single', 593, 'secondary')]

In [57]:
%%time
db.execute(text("""
select avg(balance) from bank_details where job = 'admin.';
""")).fetchone()

CPU times: total: 31.2 ms
Wall time: 28.1 ms


(Decimal('1135.8389'),)

In [61]:
%%time
db.execute(text("""
select avg(balance) from bank_view where job = 'admin.';
""")).fetchone()

CPU times: total: 0 ns
Wall time: 28.6 ms


(Decimal('1135.8389'),)